In [48]:
# -*- coding: utf-8 -*-
 
from requests_oauthlib import OAuth1Session
import json
import datetime, time, sys
from abc import ABCMeta, abstractmethod
 
CK = '8qtcWiBMV7fDxIkvYQ1BfzojW'                             # Consumer Key
CS = 'mBEMIEHxQC1oTD6tLhibAiwUbCSgc06MZFkdWVPsDpnyqEhMJh'    # Consumer Secret
AT = '514081197-235p20TizwaS8F4taqFmWWhzfxki7kXIAaUrhUgk'    # Access Token
AS = 'e6SXRAlAaAfrMLremDFZw1dV73E8Px5Ez49aIKEzPfULl'         # Accesss Token Secert
 
class TweetsGetter(object):
    __metaclass__ = ABCMeta
 
    def __init__(self):
        self.session = OAuth1Session(CK, CS, AT, AS)
 
    @abstractmethod
    def specifyUrlAndParams(self, keyword):
        '''
        呼出し先 URL、パラメータを返す
        '''
 
    @abstractmethod
    def pickupTweet(self, res_text, includeRetweet):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
 
    @abstractmethod
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
 
    def collect(self, total = -1, onlyText = False, includeRetweet = False):
        '''
        ツイート取得を開始する
        '''
 
        #----------------
        # 回数制限を確認
        #----------------
        self.checkLimit()
 
        #----------------
        # URL、パラメータ
        #----------------
        url, params = self.specifyUrlAndParams()
        params['include_rts'] = str(includeRetweet).lower()
        # include_rts は statuses/user_timeline のパラメータ。search/tweets には無効
 
        #----------------
        # ツイート取得
        #----------------
        cnt = 0
        unavailableCnt = 0
        while True:
            res = self.session.get(url, params = params)
            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)
 
                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue
 
            unavailableCnt = 0
 
            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)
 
            tweets = self.pickupTweet(json.loads(res.text))
            if len(tweets) == 0:
                # len(tweets) != params['count'] としたいが
                # count は最大値らしいので判定に使えない。
                # ⇒  "== 0" にする
                # https://dev.twitter.com/discussions/7513
                break
 
            for tweet in tweets:
                if (('retweeted_status' in tweet) and (includeRetweet is False)):
                    pass
                else:
                    if onlyText is True:
                        yield tweet['text']
                    else:
                        yield tweet
 
                    cnt += 1
                    if cnt % 100 == 0:
                        print ('%d件 ' % cnt)
 
                    if total > 0 and cnt >= total:
                        return
 
            params['max_id'] = tweet['id'] - 1
 
            # ヘッダ確認 （回数制限）
            # X-Rate-Limit-Remaining が入ってないことが稀にあるのでチェック
            if ('X-Rate-Limit-Remaining' in res.headers and 'X-Rate-Limit-Reset' in res.headers):
                if (int(res.headers['X-Rate-Limit-Remaining']) == 0):
                    self.waitUntilReset(int(res.headers['X-Rate-Limit-Reset']))
                    self.checkLimit()
            else:
                print ('not found  -  X-Rate-Limit-Remaining or X-Rate-Limit-Reset')
                self.checkLimit()
 
    def checkLimit(self):
        '''
        回数制限を問合せ、アクセス可能になるまで wait する
        '''
        unavailableCnt = 0
        while True:
            url = "https://api.twitter.com/1.1/application/rate_limit_status.json"
            res = self.session.get(url)
 
            if res.status_code == 503:
                # 503 : Service Unavailable
                if unavailableCnt > 10:
                    raise Exception('Twitter API error %d' % res.status_code)
 
                unavailableCnt += 1
                print ('Service Unavailable 503')
                self.waitUntilReset(time.mktime(datetime.datetime.now().timetuple()) + 30)
                continue
 
            unavailableCnt = 0
 
            if res.status_code != 200:
                raise Exception('Twitter API error %d' % res.status_code)
 
            remaining, reset = self.getLimitContext(json.loads(res.text))
            if (remaining == 0):
                self.waitUntilReset(reset)
            else:
                break
 
    def waitUntilReset(self, reset):
        '''
        reset 時刻まで sleep
        '''
        seconds = reset - time.mktime(datetime.datetime.now().timetuple())
        seconds = max(seconds, 0)
        print ('\n     =====================')
        print ('     == waiting %d sec ==' % seconds)
        print ('     =====================')
        sys.stdout.flush()
        time.sleep(seconds + 10)  # 念のため + 10 秒
 
    @staticmethod
    def bySearch(keyword):
        return TweetsGetterBySearch(keyword)
 
    @staticmethod
    def byUser(screen_name):
        return TweetsGetterByUser(screen_name)
 
 
class TweetsGetterBySearch(TweetsGetter):
    '''
    キーワードでツイートを検索
    '''
    def __init__(self, keyword):
        super(TweetsGetterBySearch, self).__init__()
        self.keyword = keyword
        
    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/search/tweets.json'
        params = {'q':self.keyword, 'count':100}
        return url, params
 
    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text['statuses']:
            results.append(tweet)
 
        return results
 
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['search']['/search/tweets']['remaining']
        reset     = res_text['resources']['search']['/search/tweets']['reset']
 
        return int(remaining), int(reset)
    
 
class TweetsGetterByUser(TweetsGetter):
    '''
    ユーザーを指定してツイートを取得
    '''
    def __init__(self, screen_name):
        super(TweetsGetterByUser, self).__init__()
        self.screen_name = screen_name
        
    def specifyUrlAndParams(self):
        '''
        呼出し先 URL、パラメータを返す
        '''
        url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'
        params = {'screen_name':self.screen_name, 'count':200}
        return url, params
 
    def pickupTweet(self, res_text):
        '''
        res_text からツイートを取り出し、配列にセットして返却
        '''
        results = []
        for tweet in res_text:
            results.append(tweet)
 
        return results
 
    def getLimitContext(self, res_text):
        '''
        回数制限の情報を取得 （起動時）
        '''
        remaining = res_text['resources']['statuses']['/statuses/user_timeline']['remaining']
        reset     = res_text['resources']['statuses']['/statuses/user_timeline']['reset']
 
        return int(remaining), int(reset)
 
 
if __name__ == '__main__':
 
    # キーワードで取得
    #getter = TweetsGetter.bySearch(u'渋谷')
    
    # ユーザーを指定して取得 （screen_name）
    getter = TweetsGetter.byUser('cissan_9984')
 
    cnt = 0
    f = open('text.txt', 'w') # 書き込みモードで開く
    # 2017年のだけ取得
    year = "2017"
    
    
    for tweet in getter.collect(total = 3000):
        cnt += 1
        print ('------ %d' % cnt)
        print ('{} {} {}'.format(tweet['id'], tweet['created_at'], '@'+tweet['user']['screen_name']))
        print (tweet['text'])
        
        if tweet['created_at'][-4:] == year:
            f.write(tweet['created_at'] + "\n" +tweet['text'] + "\n")

    f.close() # ファイルを閉じる
    

------ 1
964003906194833410 Thu Feb 15 05:10:16 +0000 2018 @cissan_9984
日経爆裂しててダウ先物も現物終値比＋８０ぐらいで推移してるけど
なんとなく今日のアメリカ下げそうな気がする
先がまったく見えないので本日ノーポジ日和
------ 2
963951858191495168 Thu Feb 15 01:43:27 +0000 2018 @cissan_9984
要人発言系は苦手なので
ありがたく買い戻してノーポジ
------ 3
963948945079648256 Thu Feb 15 01:31:52 +0000 2018 @cissan_9984
ドル円買い戻し！
------ 4
963931765193695232 Thu Feb 15 00:23:36 +0000 2018 @cissan_9984
ドル円どうもこのままで済むとは思えない
再度売り
------ 5
963925713509167104 Wed Feb 14 23:59:33 +0000 2018 @cissan_9984
ドル円１０６．５割れでどさくさ全部買い戻し
先物も２１３４０で買い戻し
------ 6
963609804966002688 Wed Feb 14 03:04:15 +0000 2018 @cissan_9984
うっひょー
ドル円の利益の増え方が音速
------ 7
963596371088392192 Wed Feb 14 02:10:52 +0000 2018 @cissan_9984
ドル円売り増し
------ 8
963579472581554178 Wed Feb 14 01:03:43 +0000 2018 @cissan_9984
うーん
もったいないことしたか～
１０００枚売ってたから先物の板の薄さにビビってしまった
------ 9
963574361587302400 Wed Feb 14 00:43:24 +0000 2018 @cissan_9984
とうとう売ってた日経先物を21250あたりで半分買い戻し
先物の流動性を考えて動きやすいポジションに
------ 10
963279457522925568 Tue Feb 13 05:11:34 +0000 2

------ 200
927757166399823872 Tue Nov 07 04:38:40 +0000 2017 @cissan_9984
携帯会社さらに買い増し
オプションはビビってフルカバー
プット売り状態になってしまった
200件 
------ 201
927709993234661376 Tue Nov 07 01:31:13 +0000 2017 @cissan_9984
あれ・・・
もしかして日経オプションとか一番売っちゃいけなかったやつ？？？
------ 202
927708556110962688 Tue Nov 07 01:25:30 +0000 2017 @cissan_9984
俺がコール売ったら日経の上げ加速来ちゃった・・・
金曜までは２２９００ぐらいまでで勘弁してほしい
ゆっくり行こうよ！
------ 203
927707210217484288 Tue Nov 07 01:20:09 +0000 2017 @cissan_9984
指数馬鹿上げ一服とみて日経コール２２８７５を３００枚売って先物を５０枚だけ購入
緩やかな上昇もしくはヨコヨコを望む
------ 204
927699556854259712 Tue Nov 07 00:49:44 +0000 2017 @cissan_9984
ハリケーンで道路がプールになってる動画見るたびに思うんだけど
アメリカの治水っておかしくないか？？
日本は台風来てもかすり傷すら負わない
アメリカは治水関連は日本を見習うべきだね
------ 205
927698024335261698 Tue Nov 07 00:43:39 +0000 2017 @cissan_9984
アメリカの洪水で４億円お支払い来ちゃった・・・（再保険売り）
現地の人はジェットスキーでイエーイ！して
尻ぬぐいは俺かよー
------ 206
927696780849856512 Tue Nov 07 00:38:42 +0000 2017 @cissan_9984
とある携帯会社寄りから無限買い増しで結構リバったけど
俺が買うのやめたら無限に弱くなった
これやばいやつじゃ・・・・・

それでも根性ホールドするけど
------ 207
927694570766217216 Tue Nov 07 00:29:56

------ 393
820213908581412864 Sat Jan 14 10:20:09 +0000 2017 @cissan_9984
株か為替のツイートをするとフォロワーが200人増える
ドラクエの画像を貼るとフォロワーが40人減る
毎回この現象が起こる
------ 394
820209879763718144 Sat Jan 14 10:04:08 +0000 2017 @cissan_9984
ダイヤモンドスライムかみさまにやられる！
ありがとうございました https://t.co/8XlHxJBMZi
------ 395
820198277106651136 Sat Jan 14 09:18:02 +0000 2017 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][30秒]
[対戦コード:81423215]

 #DQMSL対戦_無制限
１８時４５分頃まで募集
誰でもどうぞー
------ 396
820151929342816256 Sat Jan 14 06:13:52 +0000 2017 @cissan_9984
今日の負けー
対戦ありがとうございました！
今日から勝者はランダムで晒します https://t.co/xX0mFYyu7L
------ 397
820136314427174912 Sat Jan 14 05:11:49 +0000 2017 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][30秒]
[対戦コード:72650318]

 #DQMSL対戦_無制限
３時頃まで募集
誰でも！
------ 398
819776905037549568 Fri Jan 13 05:23:39 +0000 2017 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][30秒]
[対戦コード:05299009]

 #DQMSL対戦_無制限
素早さパーティー
３時まで募集
誰か
------ 399
819750793460871168 Fri Jan 13 03:39:54 +0000 2017 @cissan_9984
19210から損切り開始で
19260でフ

------ 588
746247692012032000 Fri Jun 24 07:44:49 +0000 2016 @cissan_9984
値動き的には15700ぐらいまで戻りもある
------ 589
746211598860091395 Fri Jun 24 05:21:23 +0000 2016 @cissan_9984
本日カジクエ行きまっす！
------ 590
746209432527220736 Fri Jun 24 05:12:47 +0000 2016 @cissan_9984
鉄火場終了感
これにて俺のザラ場監視は終わり！
------ 591
746196355043889152 Fri Jun 24 04:20:49 +0000 2016 @cissan_9984
プットも持ってたら利益になりそうだけど
想定外の値下がりと今後動きやすくするために諦めロスカットも大切
ってことで！
------ 592
746194595055206401 Fri Jun 24 04:13:49 +0000 2016 @cissan_9984
１５０００プット４９０円で俺も離脱
２４５円抜かれ・・・・・
------ 593
746192625900740608 Fri Jun 24 04:06:00 +0000 2016 @cissan_9984
なんとなくリバりそうな予感
------ 594
746177754954031104 Fri Jun 24 03:06:54 +0000 2016 @cissan_9984
後場に備えて風呂って来よ・・・
------ 595
746177172897832960 Fri Jun 24 03:04:35 +0000 2016 @cissan_9984
15100あたりで織り込み＆政策期待で買ってみたいけどいかんせん板が薄い
------ 596
746176713080508417 Fri Jun 24 03:02:46 +0000 2016 @cissan_9984
そろそろ離脱織り込んだかな・・・？
------ 597
746174489025617920 Fri Jun 24 02:53:56 +0000 2016 @cissan_9984
俺の１５０００プットやばくない？
薄くて損切れないし

------ 776
690389977662885892 Fri Jan 22 04:26:12 +0000 2016 @cissan_9984
暴落は去りました
------ 777
690061536199626752 Thu Jan 21 06:41:05 +0000 2016 @cissan_9984
@hidekin_DQMSL 
おきのどくですが
------ 778
690060563972513792 Thu Jan 21 06:37:13 +0000 2016 @cissan_9984
星ドラいいよ！
ベットの中でやってるとすぐ寝落ちできるし
まじおすすめ
------ 779
690059626809810945 Thu Jan 21 06:33:30 +0000 2016 @cissan_9984
DQMSL、、、
ドレアム１週間でオワコンかよ！
ある意味神展開
持ってないと対戦出来無いバランスはいかんなぁ
１００万かかるし
------ 780
690028481980293121 Thu Jan 21 04:29:44 +0000 2016 @cissan_9984
シャープすった。。。
------ 781
690027671955222528 Thu Jan 21 04:26:31 +0000 2016 @cissan_9984
シャープ@１４６に即被せ売りかました！
どきどき
------ 782
688893901243977728 Mon Jan 18 01:21:19 +0000 2016 @cissan_9984
毎度のこれか！
下がった時だけ高速上げ
------ 783
686778377953357825 Tue Jan 12 05:14:59 +0000 2016 @cissan_9984
今まで日銀がお金刷って株買うだけで株価吊り上げすぎた・・・
もう無駄に株買わない方がよろしいかと。
------ 784
686752220180680704 Tue Jan 12 03:31:03 +0000 2016 @cissan_9984
後場公的さんの買い
------ 785
686702672410312704 Tue Jan 12 00:14:10 +0000 2016 @cissan_9984
ホンダはまだいいと

...

------ 2149
545051815339782144 Wed Dec 17 03:04:14 +0000 2014 @cissan_9984
トヨタ買い戻して、ガンホーも買い戻して、マーベラスも買い戻して
すっきり
ガンホーは急速上げによりなぜか損切りになった。
最近は下げた後の上げ、上げた後の下げ、早すぎる！
------ 2150
545048986843770881 Wed Dec 17 02:52:59 +0000 2014 @cissan_9984
ＦＯＭＣにブルって急落したんだろ？
的な考えでＳ＆Ｐ先物を買ってみた
俺には珍しい逆張り
------ 2151
544876146307768320 Tue Dec 16 15:26:11 +0000 2014 @cissan_9984
@tgad1980 
３～１５０万ぐらいですよ
------ 2152
544363295067488256 Mon Dec 15 05:28:17 +0000 2014 @cissan_9984
@takunora 
おー黒サンタ！ｗ
よろしくね～
------ 2153
544355080544083968 Mon Dec 15 04:55:39 +0000 2014 @cissan_9984
@kijiuma_DQMSL 
大会おつ～
これからもよろしくお願いします！
クラはまじですげーな・・・
同じドラクエゲーマーとしてリスペクトせざるえない！
------ 2154
544353647249719297 Mon Dec 15 04:49:57 +0000 2014 @cissan_9984
うお、むしろクラにレベル離された！
大会よりむしろここが異次元バトル・・・
------ 2155
543796724146249731 Sat Dec 13 15:56:56 +0000 2014 @cissan_9984
らいまな関連のついーとありがとうございます！！！
１０時から何も食べられないのが大変でした。
------ 2156
543596327624392705 Sat Dec 13 02:40:38 +0000 2014 @cissan_9984
今日の３時からのニコ生のらいなまに俺も出るよ〜
------ 2157
543449595481571330

------ 2347
524956125934800896 Wed Oct 22 16:11:08 +0000 2014 @cissan_9984
@kaoken03040501 
全部斬撃にすると連携１．２倍が良く出て強烈だね～
------ 2348
524952789756018688 Wed Oct 22 15:57:52 +0000 2014 @cissan_9984
@stonecold31636 
おつー
やっぱりハゲオタがねぇ・・・
------ 2349
524952721263050752 Wed Oct 22 15:57:36 +0000 2014 @cissan_9984
@kk_shu_kai 
ぜひお願いします～
カイザーやハゲオタ＋３ガイアも１回しか出せないから
いろいろモンスターもってればチャンス多いかも
------ 2350
524951454365126656 Wed Oct 22 15:52:34 +0000 2014 @cissan_9984
@T62mccT 
もちろん！
やりますか
------ 2351
524951107391344640 Wed Oct 22 15:51:11 +0000 2014 @cissan_9984
@kk_shu_kai 
参加制限ありませんよ！
最低１５体モンスターいればおｋです
------ 2352
524948393194319873 Wed Oct 22 15:40:24 +0000 2014 @cissan_9984
@cissan_9984 
だ、誰かいないか・・・
しょぼーん
------ 2353
524947308085907456 Wed Oct 22 15:36:05 +0000 2014 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][60秒]
[対戦コード:03700197]
極悪チーム

 #DQMSL対戦_無制限
------ 2354
524940236816523266 Wed Oct 22 15:08:00 +0000 2014 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][60秒]
[対戦コード:03700197]

 #DQMSL対戦

------ 2546
516603020222529536 Mon Sep 29 14:58:52 +0000 2014 @cissan_9984
@k912000 
いや～
今日はよく状態異常入ったとはいえ、もう５割勝てなそうな予感がします・・・
------ 2547
516602019415478272 Mon Sep 29 14:54:54 +0000 2014 @cissan_9984
キングにまったく勝てねー
しすさんオワコン化した・・・
------ 2548
516600426091667456 Mon Sep 29 14:48:34 +0000 2014 @cissan_9984
@negi_hatsune @big_the_noukin 
円で、ドルで、バラモスの地図で持っているので！！！
------ 2549
516599822208335872 Mon Sep 29 14:46:10 +0000 2014 @cissan_9984
@big_the_noukin 
確率変わらず？
それなら作り直さなくていいか・・・
------ 2550
516599363687030784 Mon Sep 29 14:44:20 +0000 2014 @cissan_9984
@swgbc9991 
おお～
フレンドですねー
対戦これからもお願いします！
------ 2551
516598714161319936 Mon Sep 29 14:41:46 +0000 2014 @cissan_9984
@michandqmsl 
むしろ毎日でも！
------ 2552
516598263948906496 Mon Sep 29 14:39:58 +0000 2014 @cissan_9984
状態異常ゲー継続っぽいので
バラモスのネクロ＋付かずはエサに消えてもらって
バラモス４→１＋１、２＋２、３＋３の☆４バラモスブロスの育成にはいらざるをえない！！！
状態異常≒ザラキという認識でバランス調整してるのかねぇ・・・・・
------ 2553
516597406125682689 Mon Sep 29 14:36:34 +0000 2014 @cissan_9984
@shige19820827 @stonecold31636 
おつー
あまりにもガイ

------ 2746
503199160502210560 Sat Aug 23 15:16:43 +0000 2014 @cissan_9984
@negi_dqmsl @big_the_noukin 
ありがとー
今回はネギと脳筋さんが不参加なのがでかいね
------ 2747
503198107283099650 Sat Aug 23 15:12:32 +0000 2014 @cissan_9984
@syunm_1001 
ありがとー
また対戦しましょー
------ 2748
503198073237950464 Sat Aug 23 15:12:24 +0000 2014 @cissan_9984
@big_the_noukin 
明日から連戦ですな！
------ 2749
503197192035659776 Sat Aug 23 15:08:54 +0000 2014 @cissan_9984
@CatIlike 
しす2勝ーもっち1勝 http://t.co/WStsuL77K0
------ 2750
503193965487591428 Sat Aug 23 14:56:04 +0000 2014 @cissan_9984
@k912000 
熱戦だったね！
ありがとうございました
------ 2751
503193850941153282 Sat Aug 23 14:55:37 +0000 2014 @cissan_9984
@CatIlike 
しす3勝ーきんぐ http://t.co/P5EVbJyrbq
------ 2752
503192802600054785 Sat Aug 23 14:51:27 +0000 2014 @cissan_9984
@k912000 
もう一戦みたい
よろしくお願いします
11852278
------ 2753
503192640116887552 Sat Aug 23 14:50:48 +0000 2014 @cissan_9984
@CatIlike 
わかりました。
------ 2754
503192513755111424 Sat Aug 23 14:50:18 +0000 2014 @cissan_9984
@CatIlike 
同じ人と必ず３戦ですか？
------ 2755


------ 2946
495602393208217600 Sat Aug 02 16:09:53 +0000 2014 @cissan_9984
@satu11041 
めっちゃ強いね！
耐性もばらけてるし、いけてますな
またやりましょ～
------ 2947
495597729737756673 Sat Aug 02 15:51:21 +0000 2014 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][60秒]
[対戦コード:98507536]
誰でも！再戦も歓迎
今日は脳筋以外誰も来ない・・・さびれた！
------ 2948
495597469846077442 Sat Aug 02 15:50:19 +0000 2014 @cissan_9984
@big_the_noukin 
いつでもいいよー
------ 2949
495592526300991489 Sat Aug 02 15:30:40 +0000 2014 @cissan_9984
【ドラゴンクエストモンスターズ　スーパーライト】対戦相手募集中！
[無制限][60秒]
[対戦コード:98507536]

 #DQMSL対戦_無制限
------ 2950
495592046992711680 Sat Aug 02 15:28:46 +0000 2014 @cissan_9984
@k912000 @RayanRip777 @kanpekidonchan 
４＋４だとぎりぎりレギュラー入りだけど、それでもすぐにスタメン落ちしそう
------ 2951
495591098379542529 Sat Aug 02 15:25:00 +0000 2014 @cissan_9984
@negi_dqmsl @ryodayan 
息を吸い込む＋３でほかスルーは飲み会の時には最高おいしいけどねぇ
------ 2952
495590660326449152 Sat Aug 02 15:23:15 +0000 2014 @cissan_9984
@k912000 @RayanRip777 @kanpekidonchan 
やるじゃなくてスルーだよ
------ 2953
495588365811449856 Sat Aug 02 15:14:08

In [37]:
tweet['created_at'][-4:]

'2018'